In [1]:
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DecimalType
from pyspark.sql import functions
import json, decimal

spark = SparkSession.builder.master("local").appName("app").getOrCreate()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
56,application_1628529429212_0060,pyspark3,idle,Link,Link,✔


SparkSession available as 'spark'.


In [2]:
# conf.set is used to input your key value pairs.
spark.conf.set(
"fs.azure.account.key.adfrcdevstorage.blob.core.windows.net",
"xrbJuOZpG1+WGZk2yJhbNbaIjB+adtF6d8BN+3xLnYMEDtwA+B3fzi44qPaPNpr8OJgzFXehzBQFC4/ZqX6vCQ=="
)

# Define the location of our csv and json folders created in step 2
csv_folder = 'wasbs://guidedcapstone-2021-08-09t17-05-44-969z@guidedcapstonhdistorage.blob.core.windows.net/HdiNotebooks/csv_folder/partition=T'
json_folder = 'wasbs://guidedcapstone-2021-08-09t17-05-44-969z@guidedcapstonhdistorage.blob.core.windows.net/HdiNotebooks/json_folder/partition=T'


In [3]:
test1= spark.read.parquet('wasbs://guidedcapstone-2021-08-09t17-05-44-969z@guidedcapstonhdistorage.blob.core.windows.net/HdiNotebooks/csv_folder/partition=T/part-00000-cfdf3cfc-e3d7-4f4e-afdd-428cfc82e54c.c000.snappy.parquet')
test1.show()
test2 = spark.read.parquet('wasbs://guidedcapstone-2021-08-09t17-05-44-969z@guidedcapstonhdistorage.blob.core.windows.net/HdiNotebooks/csv_folder/partition=T/part-00001-cfdf3cfc-e3d7-4f4e-afdd-428cfc82e54c.c000.snappy.parquet')
test2.show()
trade_common_csv = spark.read.parquet(csv_folder)

trade_common_json = spark.read.parquet(json_folder)


+----------+--------------------+--------+------+--------------------+------------+-------------+------+--------+------+--------+
|  trade_dt|          arrival_tm|rec_type|symbol|            event_tm|event_seq_nb|trade_pr/exch|bid_pr|bid_size|ask_pr|ask_size|
+----------+--------------------+--------+------+--------------------+------------+-------------+------+--------+------+--------+
|2020-08-06|2020-08-06 09:30:...|       T|  SYMA|2020-08-06 10:49:...|          10|         NYSE|    74|      27|  null|    null|
|2020-08-06|2020-08-06 09:30:...|       T|  SYMA|2020-08-06 12:00:...|          20|         NYSE|    76|     315|  null|    null|
|2020-08-06|2020-08-06 09:30:...|       T|  SYMA|2020-08-06 13:11:...|          30|         NYSE|    77|     755|  null|    null|
|2020-08-06|2020-08-06 09:30:...|       T|  SYMA|2020-08-06 14:27:...|          40|         NYSE|    77|      81|  null|    null|
|2020-08-06|2020-08-06 09:30:...|       T|  SYMA|2020-08-06 15:39:...|          50|       

In [4]:
trade_csv = trade_common_csv.select('trade_dt', 'symbol', 'trade_pr/exch', 'event_tm', 'event_seq_nb','arrival_tm')
trade_csv.show()
trade_json = trade_common_json.select('trade_dt', 'symbol', 'trade_pr/exch', 'event_tm', 'event_seq_nb','arrival_tm')
trade_json.show()

+----------+------+-------------+--------------------+------------+--------------------+
|  trade_dt|symbol|trade_pr/exch|            event_tm|event_seq_nb|          arrival_tm|
+----------+------+-------------+--------------------+------------+--------------------+
|2020-08-06|  SYMB|         NYSE|2020-08-06 16:57:...|          60|2020-08-06 09:30:...|
|2020-08-06|  SYMB|         NYSE|2020-08-06 18:06:...|          70|2020-08-06 09:30:...|
|2020-08-06|  SYMB|         NYSE|2020-08-06 19:21:...|          80|2020-08-06 09:30:...|
|2020-08-06|  SYMB|         NYSE|2020-08-06 20:36:...|          90|2020-08-06 09:30:...|
|2020-08-06|  SYMB|         NYSE|2020-08-06 21:46:...|         100|2020-08-06 09:30:...|
|2020-08-06|  SYMC|         NYSE|2020-08-06 10:42:...|          10|2020-08-06 09:30:...|
|2020-08-06|  SYMC|         NYSE|2020-08-06 11:52:...|          20|2020-08-06 09:30:...|
|2020-08-06|  SYMC|         NYSE|2020-08-06 13:01:...|          30|2020-08-06 09:30:...|
|2020-08-06|  SYMC|  

In [5]:
def applyLatest(item):
    recent_trade = item.groupBy('trade_dt','Symbol','trade_pr/exch', 'event_tm', 'event_seq_nb').agg(functions.max('arrival_tm').alias('max_arrival_time'))
    return recent_trade

trade_corrected = applyLatest(trade_csv)
quote_corrected = applyLatest(trade_json)
trade_corrected.show()
quote_corrected.show()

+----------+------+-------------+--------------------+------------+--------------------+
|  trade_dt|Symbol|trade_pr/exch|            event_tm|event_seq_nb|    max_arrival_time|
+----------+------+-------------+--------------------+------------+--------------------+
|2020-08-06|  SYMA|         NYSE|2020-08-06 22:00:...|         100|2020-08-06 09:30:...|
|2020-08-06|  SYMC|         NYSE|2020-08-06 11:52:...|          20|2020-08-06 09:30:...|
|2020-08-06|  SYMC|         NYSE|2020-08-06 17:37:...|          70|2020-08-06 09:30:...|
|2020-08-06|  SYMA|         NYSE|2020-08-06 18:14:...|          70|2020-08-06 09:30:...|
|2020-08-06|  SYMA|         NYSE|2020-08-06 15:39:...|          50|2020-08-06 09:30:...|
|2020-08-06|  SYMB|         NYSE|2020-08-06 20:36:...|          90|2020-08-06 09:30:...|
|2020-08-06|  SYMB|         NYSE|2020-08-06 16:57:...|          60|2020-08-06 09:30:...|
|2020-08-06|  SYMA|         NYSE|2020-08-06 10:49:...|          10|2020-08-06 09:30:...|
|2020-08-06|  SYMB|  

In [10]:
# To save the files in our HDI cluster

trade_date = '2020-07-29'
trade_corrected.write.parquet("wasbs://guidedcapstone-2021-08-09t17-05-44-969z@guidedcapstonhdistorage.blob.core.windows.net/HdiNotebooks/trade/trade_dt={}".format(trade_date))
quote_corrected.write.parquet("wasbs://guidedcapstone-2021-08-09t17-05-44-969z@guidedcapstonhdistorage.blob.core.windows.net/HdiNotebooks/trade/quote_dt={}".format(trade_date))